In [38]:
import numpy as np
import pandas as pd
from xgboost import XGBRFRegressor, XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import mean_absolute_error, accuracy_score, mean_squared_error

In [47]:
df = pd.read_json('stevens_data_master.json')
df = df.sort_values(by='Date/Time (PST)',ascending=True)
df.tail()

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow,month
4,2019-04-30 19:00:00,13.58,37.24,0.088,9.15,20.58,283.8,2019-04-30,58.71,73.74,44.20,None,NaN,NaN,NaN,None,4
3,2019-04-30 20:00:00,12.93,36.32,5.283,12.32,16.57,276.0,2019-04-30,57.70,73.84,47.01,None,NaN,NaN,NaN,None,4
2,2019-04-30 21:00:00,12.85,35.28,8.180,12.70,17.65,274.7,2019-04-30,57.03,73.71,44.28,None,NaN,NaN,NaN,None,4
1,2019-04-30 22:00:00,12.82,34.52,4.910,11.97,16.22,275.2,2019-04-30,56.09,73.82,44.66,None,NaN,NaN,NaN,None,4
0,2019-04-30 23:00:00,12.80,33.07,5.962,9.23,13.33,267.0,2019-04-30,54.89,74.06,52.35,None,NaN,NaN,NaN,None,4


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21303 entries, 21302 to 0
Data columns (total 17 columns):
Date/Time (PST)             21303 non-null object
Battery Voltage (v)         21296 non-null float64
Temperature (deg F)         21295 non-null float64
Wind Speed Minimum (mph)    21295 non-null float64
Wind Speed Average (mph)    21295 non-null float64
Wind Speed Maximum (mph)    21295 non-null float64
Wind Direction (deg.)       19429 non-null float64
date                        21303 non-null datetime64[ns]
24 Hour Snow (in)           17685 non-null float64
Total Snow Depth (in)       17685 non-null float64
Relative Humidity (%)       178 non-null float64
area                        19682 non-null object
danger_above_treeline       19346 non-null float64
danger_below_treeline       19346 non-null float64
danger_near_treeline        19346 non-null float64
date_tomorrow               19682 non-null object
month                       21303 non-null int64
dtypes: datetime64[ns](1

### It appears that the relative humidity column is nearly empty (only 178 columns, bummer)

In [49]:
#convert datetime back into datetime format from string
df['Date/Time (PST)'] = pd.to_datetime(df['Date/Time (PST)'])

In [50]:
df.head(1)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),Relative Humidity (%),area,danger_above_treeline,danger_below_treeline,danger_near_treeline,date_tomorrow,month
21302,2014-12-17 13:00:00,0.0,12.89,25.3,0.008,0.81,125.4,2014-12-17,NaN,NaN,NaN,None,NaN,NaN,NaN,None,12


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21303 entries, 21302 to 0
Data columns (total 17 columns):
Date/Time (PST)             21303 non-null datetime64[ns]
Battery Voltage (v)         21296 non-null float64
Temperature (deg F)         21295 non-null float64
Wind Speed Minimum (mph)    21295 non-null float64
Wind Speed Average (mph)    21295 non-null float64
Wind Speed Maximum (mph)    21295 non-null float64
Wind Direction (deg.)       19429 non-null float64
date                        21303 non-null datetime64[ns]
24 Hour Snow (in)           17685 non-null float64
Total Snow Depth (in)       17685 non-null float64
Relative Humidity (%)       178 non-null float64
area                        19682 non-null object
danger_above_treeline       19346 non-null float64
danger_below_treeline       19346 non-null float64
danger_near_treeline        19346 non-null float64
date_tomorrow               19682 non-null object
month                       21303 non-null int64
dtypes: datetime

In [52]:
df = df.drop('Relative Humidity (%)',axis=1)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21303 entries, 21302 to 0
Data columns (total 16 columns):
Date/Time (PST)             21303 non-null datetime64[ns]
Battery Voltage (v)         21296 non-null float64
Temperature (deg F)         21295 non-null float64
Wind Speed Minimum (mph)    21295 non-null float64
Wind Speed Average (mph)    21295 non-null float64
Wind Speed Maximum (mph)    21295 non-null float64
Wind Direction (deg.)       19429 non-null float64
date                        21303 non-null datetime64[ns]
24 Hour Snow (in)           17685 non-null float64
Total Snow Depth (in)       17685 non-null float64
area                        19682 non-null object
danger_above_treeline       19346 non-null float64
danger_below_treeline       19346 non-null float64
danger_near_treeline        19346 non-null float64
date_tomorrow               19682 non-null object
month                       21303 non-null int64
dtypes: datetime64[ns](2), float64(11), int64(1), object(2)
memor

In [54]:
df.describe()

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),24 Hour Snow (in),Total Snow Depth (in),danger_above_treeline,danger_below_treeline,danger_near_treeline,month
count,21296.000000,21295.000000,21295.000000,21295.000000,21295.000000,19429.000000,17685.000000,17685.000000,19346.000000,19346.000000,19346.000000,21303.000000
mean,12.543848,26.028648,2.420495,6.437304,12.053108,179.496993,0.018083,91.418860,2.477618,1.954202,2.301458,4.368399
std,2.549749,78.319369,5.125817,6.052230,8.402773,99.822767,91.868543,29.605817,0.780573,0.775944,0.771484,3.969149
min,0.000000,-7999.000000,-7.743000,0.000000,0.000000,0.000000,-7999.000000,-162.700000,1.000000,1.000000,1.000000,1.000000
25%,12.610000,21.580000,0.000000,2.899000,7.212000,86.300000,0.283000,73.360000,2.000000,1.000000,2.000000,2.000000
50%,12.660000,26.780000,0.285000,4.654000,10.000000,241.600000,0.512000,91.600000,2.000000,2.000000,2.000000,3.000000
75%,12.740000,31.870000,2.877000,8.090000,14.250000,270.700000,2.425000,114.000000,3.000000,2.000000,3.000000,4.000000
max,45.980000,68.510000,53.250000,84.900000,71.790000,359.700000,166.300000,220.500000,4.000000,4.000000,4.000000,12.000000


In [55]:
df.shape

(21303, 16)

In [57]:
#df.clip changes all values below the number provided, gets rid of negatives in my case
#wind_index = df.loc[df['Wind Speed Minimum (mph)'] <= 0].index
#change all of these to zeros

## whats the max amount of snow ever recored in a day?

There are two contenders for this title. The most commonly accepted figure is the 75.8” of snow that fell in 24 hours at Silver Lake, Colorado on April 14-15, 1921. The other option is a 78.0” total measured at Mile 47 Camp, Alaska on February 7, 1963.Dec 20, 2016

This being said, lets cut if off at 75". rather than delete and lose info, lets assume machine was working and correct it to this cap we establishing

likewise for the lowest 24 hour snow period. Melting events can occur, so lets set this limit to a foot

In [60]:
df['24 Hour Snow (in)'] = df['24 Hour Snow (in)'].clip(upper=75)
df['24 Hour Snow (in)'] = df['24 Hour Snow (in)'].clip(lower=-12)
df['Total Snow Depth (in)'] = df['Total Snow Depth (in)'].clip(lower=0)
df['Temperature (deg F)'] = df['Temperature (deg F)'].clip(lower=-10)
df['Wind Speed Minimum (mph)'] = df['Wind Speed Minimum (mph)'].clip(lower=0)

In [61]:
df.shape

(21303, 16)

In [62]:
## I Need to create my boundaries of possible readings before making my rolling columns
#-24 hour min/max temp

#-48 hour min/max temp

#-24/48h/72h 24 hour snow max

#-24/48 H max wind

df['max_1_day_temp']= df['Temperature (deg F)'].rolling(24).max()
df['min_1_day_temp']= df['Temperature (deg F)'].rolling(24).min()
df['max_2_day_temp']= df['Temperature (deg F)'].rolling(48).max()
df['min_2_day_temp']= df['Temperature (deg F)'].rolling(48).min()

df['max_1_day_wind']= df['Wind Speed Maximum (mph)'].rolling(24).max()
df['max_2_day_wind']= df['Wind Speed Maximum (mph)'].rolling(48).max()

df['max_1_day_snow']= df['24 Hour Snow (in)'].rolling(24).max()
df['max_2_day_snow']= df['24 Hour Snow (in)'].rolling(48).max()
df['max_3_day_snow']= df['24 Hour Snow (in)'].rolling(72).max()

In [63]:
df.shape

(21303, 25)

In [64]:
df.head(3)

,Date/Time (PST),Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),date,24 Hour Snow (in),Total Snow Depth (in),...,month,max_1_day_temp,min_1_day_temp,max_2_day_temp,min_2_day_temp,max_1_day_wind,max_2_day_wind,max_1_day_snow,max_2_day_snow,max_3_day_snow
21302,2014-12-17 13:00:00,0.00,12.89,25.300,0.008,0.81,125.4,2014-12-17,NaN,NaN,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21301,2014-12-17 14:00:00,12.79,25.97,0.004,0.101,10.00,NaN,2014-12-17,NaN,NaN,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21300,2014-12-17 15:00:00,12.76,25.30,0.003,0.439,10.00,NaN,2014-12-17,NaN,NaN,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df.describe() #looks a lot better!

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),24 Hour Snow (in),Total Snow Depth (in),danger_above_treeline,danger_below_treeline,...,month,max_1_day_temp,min_1_day_temp,max_2_day_temp,min_2_day_temp,max_1_day_wind,max_2_day_wind,max_1_day_snow,max_2_day_snow,max_3_day_snow
count,21296.000000,21295.000000,21295.000000,21295.000000,21295.000000,19429.000000,17685.000000,17685.000000,19346.000000,19346.000000,...,21303.000000,21088.000000,21088.000000,20872.00000,20872.000000,21088.000000,20872.000000,17478.000000,17262.000000,17046.000000
mean,12.543848,26.780736,2.420865,6.437304,12.053108,179.496993,4.609215,91.431192,2.477618,1.954202,...,4.368399,31.771706,22.944213,34.07113,21.324983,20.646100,24.834413,14.036090,19.478129,23.689635
std,2.549749,9.194085,5.125367,6.052230,8.402773,99.822767,13.874658,29.539448,0.780573,0.775944,...,3.969149,9.829899,8.194052,9.72274,7.982558,11.564135,12.607514,20.977083,22.588715,22.962622
min,0.000000,-10.000000,0.000000,0.000000,0.000000,0.000000,-12.000000,0.000000,1.000000,1.000000,...,1.000000,0.066000,-10.000000,1.00800,-10.000000,0.000000,0.000000,-12.000000,-12.000000,-12.000000
25%,12.610000,21.580000,0.000000,2.899000,7.212000,86.300000,0.283000,73.360000,2.000000,1.000000,...,2.000000,25.640000,18.420000,27.72000,16.920000,11.750000,14.875000,0.614000,1.215000,2.796000
50%,12.660000,26.780000,0.285000,4.654000,10.000000,241.600000,0.512000,91.600000,2.000000,2.000000,...,3.000000,31.290000,23.630000,33.18000,22.540000,17.340000,22.930000,2.787000,5.388000,10.730000
75%,12.740000,31.870000,2.877000,8.090000,14.250000,270.700000,2.425000,114.000000,3.000000,2.000000,...,4.000000,37.530000,28.130000,40.14000,26.340000,27.160000,32.405000,14.110000,46.490000,46.490000
max,45.980000,68.510000,53.250000,84.900000,71.790000,359.700000,75.000000,220.500000,4.000000,4.000000,...,12.000000,68.510000,53.960000,68.51000,49.480000,71.790000,71.790000,75.000000,75.000000,75.000000


In [66]:
df.columns

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Temperature (deg F)',
       'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
       'Wind Speed Maximum (mph)', 'Wind Direction (deg.)', 'date',
       '24 Hour Snow (in)', 'Total Snow Depth (in)', 'area',
       'danger_above_treeline', 'danger_below_treeline',
       'danger_near_treeline', 'date_tomorrow', 'month', 'max_1_day_temp',
       'min_1_day_temp', 'max_2_day_temp', 'min_2_day_temp', 'max_1_day_wind',
       'max_2_day_wind', 'max_1_day_snow', 'max_2_day_snow', 'max_3_day_snow'],
      dtype='object')

In [67]:
df.isna().sum()

Date/Time (PST)                0
Battery Voltage (v)            7
Temperature (deg F)            8
Wind Speed Minimum (mph)       8
Wind Speed Average (mph)       8
Wind Speed Maximum (mph)       8
Wind Direction (deg.)       1874
date                           0
24 Hour Snow (in)           3618
Total Snow Depth (in)       3618
area                        1621
danger_above_treeline       1957
danger_below_treeline       1957
danger_near_treeline        1957
date_tomorrow               1621
month                          0
max_1_day_temp               215
min_1_day_temp               215
max_2_day_temp               431
min_2_day_temp               431
max_1_day_wind               215
max_2_day_wind               431
max_1_day_snow              3825
max_2_day_snow              4041
max_3_day_snow              4257
dtype: int64

In [68]:
no_avy_index = df.loc[df['danger_above_treeline'].isna()].index
df = df.drop(index=no_avy_index,axis=0)

In [69]:
df.isna().sum()

#wind direction and 23 hour snow depth must also be removed


Date/Time (PST)                0
Battery Voltage (v)            5
Temperature (deg F)            6
Wind Speed Minimum (mph)       6
Wind Speed Average (mph)       6
Wind Speed Maximum (mph)       6
Wind Direction (deg.)       1838
date                           0
24 Hour Snow (in)           3220
Total Snow Depth (in)       3220
area                           0
danger_above_treeline          0
danger_below_treeline          0
danger_near_treeline           0
date_tomorrow                  0
month                          0
max_1_day_temp               144
min_1_day_temp               144
max_2_day_temp               300
min_2_day_temp               300
max_1_day_wind               144
max_2_day_wind               300
max_1_day_snow              3335
max_2_day_snow              3455
max_3_day_snow              3575
dtype: int64

In [70]:
no_snow_depth = df.loc[df['Total Snow Depth (in)'].isna()].index

In [71]:
df.loc[df['Total Snow Depth (in)'].isna()].shape[0]

3220

In [72]:
df.shape[0] - df.drop(index=no_snow_depth).shape[0]

3220

In [73]:
df = df.drop(index=no_snow_depth)

In [74]:
df.isna().sum()


Date/Time (PST)               0
Battery Voltage (v)           0
Temperature (deg F)           1
Wind Speed Minimum (mph)      1
Wind Speed Average (mph)      1
Wind Speed Maximum (mph)      1
Wind Direction (deg.)         1
date                          0
24 Hour Snow (in)             0
Total Snow Depth (in)         0
area                          0
danger_above_treeline         0
danger_below_treeline         0
danger_near_treeline          0
date_tomorrow                 0
month                         0
max_1_day_temp              139
min_1_day_temp              139
max_2_day_temp              283
min_2_day_temp              283
max_1_day_wind              139
max_2_day_wind              283
max_1_day_snow              115
max_2_day_snow              235
max_3_day_snow              355
dtype: int64

In [75]:
df_clean = df.dropna()

## Now, what do i want my X's to be?? 

X: temp, wind speed average, 24 hour snow (in), max/min 24 and 48 H temp 

Y: danger_near_treeline

In [76]:
df_clean = df_clean.reset_index(drop=True)

In [77]:
df_clean.columns

Index(['Date/Time (PST)', 'Battery Voltage (v)', 'Temperature (deg F)',
       'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
       'Wind Speed Maximum (mph)', 'Wind Direction (deg.)', 'date',
       '24 Hour Snow (in)', 'Total Snow Depth (in)', 'area',
       'danger_above_treeline', 'danger_below_treeline',
       'danger_near_treeline', 'date_tomorrow', 'month', 'max_1_day_temp',
       'min_1_day_temp', 'max_2_day_temp', 'min_2_day_temp', 'max_1_day_wind',
       'max_2_day_wind', 'max_1_day_snow', 'max_2_day_snow', 'max_3_day_snow'],
      dtype='object')

In [78]:
df_clean.shape[0]*.80


12578.400000000001

In [90]:
X = df_clean[[
            'Battery Voltage (v)', 'Temperature (deg F)',
            'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
            'Wind Speed Maximum (mph)', 'Wind Direction (deg.)',
            '24 Hour Snow (in)', 'Total Snow Depth (in)','max_1_day_temp',
            'min_1_day_temp', 'max_2_day_temp', 'min_2_day_temp',
            'max_1_day_snow', 'max_2_day_snow', 'max_3_day_snow']]

y = df_clean['danger_near_treeline']

In [92]:
X_train, X_test, y_train, y_test = X[:12578], X[12578:], y[:12578], y[12578:]

DO A CHRONOLIGCAL SPLIT, because its a time series problem


In [93]:
X_train.isna().sum()

Battery Voltage (v)         0
Temperature (deg F)         0
Wind Speed Minimum (mph)    0
Wind Speed Average (mph)    0
Wind Speed Maximum (mph)    0
Wind Direction (deg.)       0
24 Hour Snow (in)           0
Total Snow Depth (in)       0
max_1_day_temp              0
min_1_day_temp              0
max_2_day_temp              0
min_2_day_temp              0
max_1_day_snow              0
max_2_day_snow              0
max_3_day_snow              0
dtype: int64

In [94]:
ss = StandardScaler()

In [99]:
mapper = DataFrameMapper([
    ([   'Battery Voltage (v)', 'Temperature (deg F)',
            'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
            'Wind Speed Maximum (mph)', 'Wind Direction (deg.)',
            '24 Hour Snow (in)', 'Total Snow Depth (in)','max_1_day_temp',
            'min_1_day_temp', 'max_2_day_temp', 'min_2_day_temp',
            'max_1_day_snow', 'max_2_day_snow', 'max_3_day_snow'], ss)
    ])

In [100]:
lr = LinearRegression()
rfr = RandomForestRegressor(n_estimators=100)
gbr = GradientBoostingRegressor(n_estimators=100)
knnr = KNeighborsRegressor()
xgbr = XGBRegressor(objective='reg:squarederror')
xgbrfr = XGBRFRegressor(objective='reg:squarederror')
regressors = [lr, rfr, gbr, knnr, xgbr,xgbrfr ]

In [101]:
reg_names = ['LinearRegression',
             'RandomForestRegressor',
             'GradientBoostingRegressor',
             'KNeighborsRegressor',
             'XGBRegressor',
             'XGBRFRegressor'
            ]

In [138]:
baseline_pred

2.4501510574018126

In [102]:
baseline_pred = np.mean(y_train)
residuals = (y_test - baseline_pred)
median_absolute_error = np.median(np.abs(residuals))
rmse=np.sqrt(np.mean(residuals**2))

print(median_absolute_error,rmse)

print(f'The MAE of just predicting the mean'
          f' is: {median_absolute_error}\n'
          f'The RMSE just predicting the mean'
          f' is: {rmse}\n'
      '==================================================================\n'
          '========================\n')

for regressor,name in zip(regressors,reg_names): #zip returns a tuple for each pair it is given
    pipe = Pipeline(steps=[
        ('scaler',mapper),
        ('regressor', regressor)
    ])
    pipe.fit(X_train, y_train)
    preds = pipe.predict(X_test)
    print(f'The MAE of the {name}'
          f' is: {mean_absolute_error((y_test),preds)}\n'
          f'The RMSE of the {name}'
          f' is: {mean_squared_error((y_test),preds)}\n'
          '==================================================================\n'
          '========================\n'
         )
    


0.5498489425981874 0.8711822606461803
The MAE of just predicting the mean is: 0.5498489425981874
The RMSE just predicting the mean is: 0.8711822606461803

The MAE of the LinearRegression is: 0.6993660636487984
The RMSE of the LinearRegression is: 0.6606762784265351

The MAE of the RandomForestRegressor is: 0.6701109754139166
The RMSE of the RandomForestRegressor is: 0.6397837293429061

The MAE of the GradientBoostingRegressor is: 0.6500092786526636
The RMSE of the GradientBoostingRegressor is: 0.5943602053289719

The MAE of the KNeighborsRegressor is: 0.7732909379968204
The RMSE of the KNeighborsRegressor is: 0.9019268680445153



/Users/krex/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/krex/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


The MAE of the XGBRegressor is: 0.6520154909412887
The RMSE of the XGBRegressor is: 0.6025043157302086



/Users/krex/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/krex/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


The MAE of the XGBRFRegressor is: 0.6779243150841451
The RMSE of the XGBRFRegressor is: 0.6110299710068424



In [105]:
pipe2 = Pipeline(steps=[
    ('scaler',mapper),
    ('regressor', gbr)
])
pipe2.fit(X_train, y_train)
preds = pipe2.predict(X_test)
print(f'The MAE of the {name}'
      f' is: {mean_absolute_error((y_test),preds)}\n'
      f'The RMSE of the {name}'
      f' is: {mean_squared_error((y_test),preds)}\n'
      '==================================================================\n'
      '========================\n'
     )



The MAE of the XGBRFRegressor is: 0.6500895531829036
The RMSE of the XGBRFRegressor is: 0.594399504533469



In [103]:
y_test[0:20]

12578    3.0
12579    2.0
12580    2.0
12581    2.0
12582    2.0
12583    2.0
12584    2.0
12585    2.0
12586    2.0
12587    2.0
12588    2.0
12589    2.0
12590    2.0
12591    2.0
12592    2.0
12593    2.0
12594    2.0
12595    2.0
12596    2.0
12597    2.0
Name: danger_near_treeline, dtype: float64

In [106]:
pipe2.predict(X_test)[0:20]

array([2.92932684, 3.00455071, 3.02375077, 3.07914031, 2.99488675,
       2.80481734, 2.63387034, 2.65692345, 2.64566369, 2.75802418,
       2.75170707, 2.74559618, 2.63973064, 2.69374685, 2.69564583,
       2.74558946, 2.74558946, 2.7077744 , 2.64901942, 2.66334031])

## Model 4 
Gradient Boosting RMSE is better than guessing the mean!

Also my model is predicting hourly avalanche forecasts, which is super granular. 

The forecasts are given daily, and thus my Y im predicting and basing my scoring on isn't as granular.

In [107]:
bar_p = pd.read_csv('csv_files/stevens_pass_barometric_pressure_2014_now.csv')

In [108]:
bar_p.head()

,Date/Time (PST),mb - 2700' - Berne,mb - 3950' - Stevens Pass - Schmidt Haus
0,2019-06-28 12:00,1017.0,1018.0
1,2019-06-28 11:00,1016.7,1018.0
2,2019-06-28 10:00,1016.6,1019.0
3,2019-06-28 09:00,1016.9,1018.0
4,2019-06-28 08:00,1017.1,1018.0


In [109]:
bar_p.describe()

,mb - 2700' - Berne,mb - 3950' - Stevens Pass - Schmidt Haus
count,39901.000000,40818.000000
mean,1014.581390,1015.308593
std,6.507781,6.392107
min,986.310000,987.000000
25%,1011.300000,1012.000000
50%,1015.100000,1016.000000
75%,1018.500000,1019.000000
max,1037.300000,1036.000000


In [110]:
bar_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 3 columns):
Date/Time (PST)                             41700 non-null object
mb - 2700' - Berne                          39901 non-null float64
mb - 3950' - Stevens Pass - Schmidt Haus    40818 non-null float64
dtypes: float64(2), object(1)
memory usage: 977.4+ KB


In [111]:
bar_p['Date/Time (PST)'] = pd.to_datetime(bar_p["Date/Time (PST)"])

In [112]:
bar_p.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41700 entries, 0 to 41699
Data columns (total 3 columns):
Date/Time (PST)                             41700 non-null datetime64[ns]
mb - 2700' - Berne                          39901 non-null float64
mb - 3950' - Stevens Pass - Schmidt Haus    40818 non-null float64
dtypes: datetime64[ns](1), float64(2)
memory usage: 977.4 KB


In [113]:
bar_p = bar_p[["Date/Time (PST)","mb - 3950\' - Stevens Pass - Schmidt Haus"]]

In [114]:
df_merge = pd.merge(df_clean,bar_p,how='left',on='Date/Time (PST)')

In [115]:
df_merge = df_merge.dropna()

In [116]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15717 entries, 0 to 15722
Data columns (total 26 columns):
Date/Time (PST)                             15717 non-null datetime64[ns]
Battery Voltage (v)                         15717 non-null float64
Temperature (deg F)                         15717 non-null float64
Wind Speed Minimum (mph)                    15717 non-null float64
Wind Speed Average (mph)                    15717 non-null float64
Wind Speed Maximum (mph)                    15717 non-null float64
Wind Direction (deg.)                       15717 non-null float64
date                                        15717 non-null datetime64[ns]
24 Hour Snow (in)                           15717 non-null float64
Total Snow Depth (in)                       15717 non-null float64
area                                        15717 non-null object
danger_above_treeline                       15717 non-null float64
danger_below_treeline                       15717 non-null float64
danger_

In [117]:
precip = pd.read_csv('csv_files/stevens_pass_precipitation_2014_now.csv')
precip['Date/Time (PST)'] = pd.to_datetime(precip["Date/Time (PST)"])

In [118]:
precip.columns = ['Date/Time (PST)', '2700ft_Berne','3950ft_SchmidtHaus','4800ft_Brooks_precip']

In [119]:
precip.head()

,Date/Time (PST),2700ft_Berne,3950ft_SchmidtHaus,4800ft_Brooks_precip
0,2019-06-28 12:00:00,0.0,0.0,NaN
1,2019-06-28 11:00:00,0.0,0.0,NaN
2,2019-06-28 10:00:00,0.0,0.0,NaN
3,2019-06-28 09:00:00,0.0,0.0,NaN
4,2019-06-28 08:00:00,0.0,0.0,NaN


In [120]:
precip.columns

Index(['Date/Time (PST)', '2700ft_Berne', '3950ft_SchmidtHaus',
       '4800ft_Brooks_precip'],
      dtype='object')

In [121]:
precip_brooks = precip[['Date/Time (PST)','4800ft_Brooks_precip']]

In [122]:
shot_snow = pd.read_csv('csv_files/stevens-pass_intermittent_snow.csv')
shot_snow.head()

,Date/Time (PST),""" - 4790' - Stevens Pass - Grace Lakes"
0,2019-06-28 12:00,-7999.0
1,2019-06-28 11:00,-7999.0
2,2019-06-28 10:00,-7999.0
3,2019-06-28 09:00,-7999.0
4,2019-06-28 08:00,-7999.0


In [123]:
shot_snow["Date/Time (PST)"] = pd.to_datetime(shot_snow["Date/Time (PST)"])

In [126]:
df_merge2 = pd.merge(df_merge,shot_snow,on="Date/Time (PST)")

In [127]:
merge_final = pd.merge(df_merge2,precip_brooks,on="Date/Time (PST)")

In [128]:
merge_final.isna().sum()

Date/Time (PST)                                0
Battery Voltage (v)                            0
Temperature (deg F)                            0
Wind Speed Minimum (mph)                       0
Wind Speed Average (mph)                       0
Wind Speed Maximum (mph)                       0
Wind Direction (deg.)                          0
date                                           0
24 Hour Snow (in)                              0
Total Snow Depth (in)                          0
area                                           0
danger_above_treeline                          0
danger_below_treeline                          0
danger_near_treeline                           0
date_tomorrow                                  0
month                                          0
max_1_day_temp                                 0
min_1_day_temp                                 0
max_2_day_temp                                 0
min_2_day_temp                                 0
max_1_day_wind      

In [129]:
merge_final.describe()

,Battery Voltage (v),Temperature (deg F),Wind Speed Minimum (mph),Wind Speed Average (mph),Wind Speed Maximum (mph),Wind Direction (deg.),24 Hour Snow (in),Total Snow Depth (in),danger_above_treeline,danger_below_treeline,...,max_2_day_temp,min_2_day_temp,max_1_day_wind,max_2_day_wind,max_1_day_snow,max_2_day_snow,max_3_day_snow,mb - 3950' - Stevens Pass - Schmidt Haus,""" - 4790' - Stevens Pass - Grace Lakes",4800ft_Brooks_precip
count,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,...,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,14824.000000,13408.000000,14824.000000
mean,12.710436,25.514694,1.545698,5.787864,11.984340,179.211941,3.332867,92.447801,2.548907,2.059431,...,32.345657,20.373483,21.031461,25.368218,12.729316,18.198232,22.275397,1012.733675,-78.421844,0.013131
std,0.221036,8.317783,2.501591,4.832151,8.405024,97.980723,10.035249,28.541649,0.766487,0.769302,...,8.893586,7.500582,11.244975,12.016986,18.715538,20.847514,21.516754,8.184510,855.590527,0.175066
min,12.430000,-2.007000,0.000000,0.000000,0.000000,0.000000,-12.000000,0.000000,1.000000,1.000000,...,6.934000,-2.007000,0.000000,0.000000,-2.916000,-1.043000,0.386000,987.000000,-7999.000000,0.000000
25%,12.600000,20.790000,0.000000,2.862000,6.905000,86.400000,0.291000,74.140000,2.000000,2.000000,...,26.562500,16.230000,12.580000,15.780000,0.614000,1.154000,2.628000,1008.000000,2.528000,0.000000
50%,12.660000,25.560000,0.088000,4.361000,9.820000,233.200000,0.507000,92.400000,3.000000,2.000000,...,31.690000,21.150000,17.710000,23.370000,2.670000,5.253000,9.010000,1014.000000,6.709000,0.000000
75%,12.730000,30.450000,2.280000,7.221250,14.345000,270.100000,2.290000,114.100000,3.000000,3.000000,...,37.430000,25.140000,27.290000,32.700000,12.090000,46.490000,46.490000,1018.000000,14.580000,0.010000
max,14.670000,67.010000,21.240000,35.030000,71.790000,358.600000,75.000000,220.500000,4.000000,4.000000,...,68.510000,48.340000,71.790000,71.790000,75.000000,75.000000,75.000000,1036.000000,511.400000,21.090000


In [130]:
merge_final['24 Hour Snow (in)'].plot();

In [131]:
merge_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14824 entries, 0 to 14823
Data columns (total 28 columns):
Date/Time (PST)                             14824 non-null datetime64[ns]
Battery Voltage (v)                         14824 non-null float64
Temperature (deg F)                         14824 non-null float64
Wind Speed Minimum (mph)                    14824 non-null float64
Wind Speed Average (mph)                    14824 non-null float64
Wind Speed Maximum (mph)                    14824 non-null float64
Wind Direction (deg.)                       14824 non-null float64
date                                        14824 non-null datetime64[ns]
24 Hour Snow (in)                           14824 non-null float64
Total Snow Depth (in)                       14824 non-null float64
area                                        14824 non-null object
danger_above_treeline                       14824 non-null float64
danger_below_treeline                       14824 non-null float64
danger_

In [133]:
mapper = DataFrameMapper([
        ([   
            'Battery Voltage (v)', 'Temperature (deg F)',
            'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
            'Wind Speed Maximum (mph)', 'Wind Direction (deg.)',
            '24 Hour Snow (in)', 'Total Snow Depth (in)','max_1_day_temp',
            'min_1_day_temp', 'max_2_day_temp', 'min_2_day_temp',
            'max_1_day_snow', 'max_2_day_snow', 'max_3_day_snow',
            "mb - 3950' - Stevens Pass - Schmidt Haus",
            '4800ft_Brooks_precip'], ss)
    ])

In [134]:
X = merge_final[[
        'Battery Voltage (v)', 'Temperature (deg F)',
            'Wind Speed Minimum (mph)', 'Wind Speed Average (mph)',
            'Wind Speed Maximum (mph)', 'Wind Direction (deg.)',
            '24 Hour Snow (in)', 'Total Snow Depth (in)','max_1_day_temp',
            'min_1_day_temp', 'max_2_day_temp', 'min_2_day_temp',
            'max_1_day_snow', 'max_2_day_snow', 'max_3_day_snow',
            "mb - 3950' - Stevens Pass - Schmidt Haus",
            '4800ft_Brooks_precip']]

y = merge_final['danger_near_treeline']

In [135]:
X_train, X_test, y_train, y_test = X[:12553], X[12553:], y[:12553], y[12553:]

In [137]:
pipe3 = Pipeline(steps=[
    ('scaler',mapper),
    ('regressor', gbr)
])
pipe3.fit(X_train, y_train)
preds = pipe3.predict(X_test)
print(f'The MAE of the {name}'
      f' is: {mean_absolute_error((y_test),preds)}\n'
      f'The RMSE of the {name}'
      f' is: {mean_squared_error((y_test),preds)}\n'
      '==================================================================\n'
      '========================\n'
     )


The MAE of the XGBRFRegressor is: 0.7295585019837179
The RMSE of the XGBRFRegressor is: 0.7324887753676314



## Conclusion
-adding precipitation and the barometric pressure makes my model worse, ha